# Pretrain

In [2]:
import os, sys, configparser, logging, argparse
sys.path.append('/'.join(os.getcwd().split('/')[:-2]))

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.optim as optim

from XAE.dataset import rmMNIST
from XAE.util import init_params

In [1]:
from model.sswae_hsic_multi import SSWAE_HSIC_MNIST as cur_model

In [67]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

d = 64
embed_data = nn.Sequential(
    nn.Conv2d(1, d, kernel_size = 4, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(d),
    nn.ReLU(True),

    nn.Conv2d(d, d, kernel_size = 4, padding = 'same', bias = False),
    nn.BatchNorm2d(d),
    nn.ReLU(True),

    nn.Conv2d(d, 2*d, kernel_size = 4, stride = 2, padding = 1, bias = False),
    nn.BatchNorm2d(2*d),
    nn.ReLU(True),

    nn.Conv2d(2*d, 2*d, kernel_size = 4, padding = 'same', bias = False),
    nn.BatchNorm2d(2*d),
    nn.ReLU(True),

    nn.Flatten(),
).to(device)

embed_condition = nn.Sequential(
    nn.Linear(49*2*d, d),
    nn.BatchNorm1d(d),
    nn.ReLU(True),

    nn.Linear(d, 10),

#     nn.Softmax(dim = 1)
).to(device)

init_params(embed_data)
init_params(embed_condition)

In [21]:
train_data = rmMNIST('./data/MNIST', train = True, label = True, aux = [[i for i in range(10)], []], portion = 0.5)
train_generator = torch.utils.data.DataLoader(train_data, 100, num_workers = 5, shuffle = True, pin_memory=True, drop_last=True)

In [68]:
epoch = 10
opt = optim.Adam(list(embed_data.parameters()) + list(embed_condition.parameters()), lr = 1e-3, betas = (0.9, 0.999))
crit = nn.CrossEntropyLoss()

running_loss = 0.0
for k in range(epoch):
    for i, (data, condition) in enumerate(train_generator):
        opt.zero_grad()
        x = data.to(device)
        y = condition.to(device)
        output = embed_condition(embed_data(x))
        loss = crit(output, y)
        loss.backward()
        opt.step()
        running_loss += loss.item()
        
    print('[%d] loss: %.3f' % (k + 1, running_loss / len(train_generator)))
    running_loss = 0.0

[1] loss: 0.843
[2] loss: 0.070
[3] loss: 0.034
[4] loss: 0.020
[5] loss: 0.014
[6] loss: 0.009
[7] loss: 0.011
[8] loss: 0.006
[9] loss: 0.004
[10] loss: 0.006


In [80]:
torch.save(embed_data.state_dict(), './result/embed_data_weight.pt')
torch.save(embed_condition.state_dict(), './result/embed_condition_weight.pt')

In [81]:
ec = nn.Sequential(
    nn.Linear(49*2*d, d),
    nn.BatchNorm1d(d),
    nn.ReLU(True),

    nn.Linear(d, 10),
    nn.Softmax(dim = 1),
).to(device)

In [82]:
ec.load_state_dict(torch.load('embed_condition_weight.pt'))

<All keys matched successfully>

In [83]:
[p for p in embed_condition.parameters()]

[Parameter containing:
 tensor([[-0.0155,  0.0008,  0.0149,  ..., -0.0441, -0.0403,  0.0297],
         [ 0.0026,  0.0034, -0.0018,  ...,  0.0015,  0.0241,  0.0109],
         [ 0.0509,  0.0307, -0.0015,  ..., -0.0055, -0.0539, -0.0012],
         ...,
         [-0.0139, -0.0003,  0.0101,  ...,  0.0207, -0.0381,  0.0005],
         [-0.0239, -0.0210,  0.0214,  ...,  0.0096,  0.0050, -0.0426],
         [-0.0404, -0.0313,  0.0009,  ...,  0.0270,  0.0156, -0.0291]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([0.1189, 0.1034, 0.1633, 0.1320, 0.1255, 0.1592, 0.1107, 0.1091, 0.1105,
         0.1454, 0.1825, 0.1306, 0.1469, 0.1633, 0.1856, 0.1190, 0.1743, 0.1177,
         0.0992, 0.1990, 0.1398, 0.1282, 0.1504, 0.1735, 0.1517, 0.1363, 0.1139,
         0.1450, 0.1223, 0.1706, 0.1824, 0.0932, 0.1710, 0.1122, 0.1615, 0.1108,
         0.1997, 0.1757, 0.1300, 0.1542, 0.0900, 0.1307, 0.1065, 0.1549, 0.1352,
         0.1342, 0.1378, 0.1257, 0.1059, 0.1366, 0.1004, 0.128

In [84]:
[p for p in ec.parameters()]

[Parameter containing:
 tensor([[-0.0155,  0.0008,  0.0149,  ..., -0.0441, -0.0403,  0.0297],
         [ 0.0026,  0.0034, -0.0018,  ...,  0.0015,  0.0241,  0.0109],
         [ 0.0509,  0.0307, -0.0015,  ..., -0.0055, -0.0539, -0.0012],
         ...,
         [-0.0139, -0.0003,  0.0101,  ...,  0.0207, -0.0381,  0.0005],
         [-0.0239, -0.0210,  0.0214,  ...,  0.0096,  0.0050, -0.0426],
         [-0.0404, -0.0313,  0.0009,  ...,  0.0270,  0.0156, -0.0291]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([0.1189, 0.1034, 0.1633, 0.1320, 0.1255, 0.1592, 0.1107, 0.1091, 0.1105,
         0.1454, 0.1825, 0.1306, 0.1469, 0.1633, 0.1856, 0.1190, 0.1743, 0.1177,
         0.0992, 0.1990, 0.1398, 0.1282, 0.1504, 0.1735, 0.1517, 0.1363, 0.1139,
         0.1450, 0.1223, 0.1706, 0.1824, 0.0932, 0.1710, 0.1122, 0.1615, 0.1108,
         0.1997, 0.1757, 0.1300, 0.1542, 0.0900, 0.1307, 0.1065, 0.1549, 0.1352,
         0.1342, 0.1378, 0.1257, 0.1059, 0.1366, 0.1004, 0.128

# Train

### dim(Pz) = 8

In [3]:
from XAE.dataset import MNIST
from XAE.util import init_params, calculate_sharpness
from XAE.sampler import gaus
from XAE.logging_daily import logging_daily
import XAE.fid as fid

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from importlib import import_module
from random import sample

In [4]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

logger = logging_daily('./config/log_info.yaml')
log = logger.get_logging()
log.setLevel(logging.INFO)

cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_multi_1.cfg')

['./config/train_config_sswae_hsic_multi_1.cfg']

In [14]:
from model.sswae_hsic_multi import SSWAE_HSIC_MNIST as cur_model

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 8
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.218071e-01  D: 6.917411e-01  D2: 7.550081e-05  D3: 5.898092e-04


[default |INFO|model.py:439] [02/30]  loss: 1.992794e-01  D: 7.627847e-01  D2: -5.222162e-05  D3: 1.203258e-02


[default |INFO|model.py:439] [03/30]  loss: 1.658297e-01  D: 6.931385e-01  D2: -5.305370e-04  D3: 2.819160e-03


[default |INFO|model.py:439] [04/30]  loss: 1.457132e-01  D: 6.931455e-01  D2: 5.058289e-04  D3: 2.073401e-03


...

[default |INFO|model.py:608] [47/50]  loss: 5.281271e-02  D: 6.931472e-01  D2: -2.891175e-03  D3: 4.936222e-04
test loss: 5.422725e-02  D: 6.931472e-01  D2: -9.808645e-03  D3: 4.891790e-04


[default |INFO|model.py:608] [48/50]  loss: 5.312695e-02  D: 6.931472e-01  D2: -2.741927e-03  D3: 4.863218e-04
test loss: 5.440860e-02  D: 6.931472e-01  D2: -9.456654e-03  D3: 4.651457e-04


[default |INFO|model.py:608] [49/50]  loss: 5.303166e-02  D: 6.931472e-01  D2: -3.003106e-03  D3: 4.857381e-04
test loss: 5.763222e-02  D: 6.931472e-01  D2: -8.897638e-03  D3: 4.954069e-04


[default |INFO|model.py:608] [50/50]  loss: 5.300373e-02  D: 6.931472e-01  D2: -2.533666e-03  D3: 4.875354e-04
test loss: 5.246836e-02  D: 6.931472e-01  D2: -9.576120e-03  D3: 4.741693e-04


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1217.674s


### dim(Pz) = 10

In [15]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_multi_2.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 10
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mo

[default |INFO|model.py:439] [01/30]  loss: 2.231114e-01  D: 6.927461e-01  D2: -4.973412e-05  D3: 3.391719e-03


[default |INFO|model.py:439] [02/30]  loss: 1.616423e-01  D: 6.925156e-01  D2: -1.885144e-04  D3: 5.875801e-03


[default |INFO|model.py:439] [03/30]  loss: 1.373740e-01  D: 6.930239e-01  D2: 8.934975e-05  D3: 5.823175e-03


[default |INFO|model.py:439] [04/30]  loss: 1.242852e-01  D: 7.011980e-01  D2: -2.891970e-04  D3: 4.339495e-03


...

[default |INFO|model.py:608] [47/50]  loss: 4.914737e-02  D: 6.950588e-01  D2: -2.036457e-03  D3: 1.487790e-02
test loss: 5.082067e-02  D: 6.826953e-01  D2: -8.712225e-03  D3: 1.531107e-02


[default |INFO|model.py:608] [48/50]  loss: 4.927014e-02  D: 6.941568e-01  D2: -2.614745e-03  D3: 1.483650e-02
test loss: 5.077395e-02  D: 6.857992e-01  D2: -9.300661e-03  D3: 1.444966e-02


[default |INFO|model.py:608] [49/50]  loss: 4.906042e-02  D: 6.937808e-01  D2: -2.590652e-03  D3: 1.494789e-02
test loss: 5.222148e-02  D: 7.085318e-01  D2: -9.138565e-03  D3: 1.531144e-02


[default |INFO|model.py:608] [50/50]  loss: 4.744046e-02  D: 6.933504e-01  D2: -2.519943e-03  D3: 1.490833e-02
test loss: 5.050907e-02  D: 6.839484e-01  D2: -9.622374e-03  D3: 1.563592e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1138.230s


### dim(Pz) = 11

In [16]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_multi_5.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 11
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mo

[default |INFO|model.py:439] [01/30]  loss: 2.224151e-01  D: 6.937247e-01  D2: -1.005014e-05  D3: 2.554138e-03


[default |INFO|model.py:439] [02/30]  loss: 1.735821e-01  D: 7.477978e-01  D2: 2.601417e-04  D3: 1.265147e-02


[default |INFO|model.py:439] [03/30]  loss: 1.773796e-01  D: 7.409029e-01  D2: 7.383760e-04  D3: 4.103957e-02


[default |INFO|model.py:439] [04/30]  loss: 1.410271e-01  D: 6.965059e-01  D2: -2.214479e-04  D3: 3.687362e-02


...

[default |INFO|model.py:608] [47/50]  loss: 4.569119e-02  D: 6.940201e-01  D2: -2.529961e-03  D3: 1.497734e-02
test loss: 4.675688e-02  D: 6.943727e-01  D2: -8.824482e-03  D3: 1.561696e-02


[default |INFO|model.py:608] [48/50]  loss: 4.545779e-02  D: 6.930258e-01  D2: -2.524827e-03  D3: 1.454320e-02
test loss: 4.798817e-02  D: 6.887788e-01  D2: -9.220686e-03  D3: 1.480624e-02


[default |INFO|model.py:608] [49/50]  loss: 4.627934e-02  D: 6.931471e-01  D2: -2.451404e-03  D3: 1.459156e-02
test loss: 4.862240e-02  D: 6.906391e-01  D2: -9.056177e-03  D3: 1.519659e-02


[default |INFO|model.py:608] [50/50]  loss: 4.643993e-02  D: 6.938020e-01  D2: -2.529372e-03  D3: 1.482144e-02
test loss: 4.787397e-02  D: 7.003950e-01  D2: -8.862534e-03  D3: 1.508658e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1147.653s


### dim(Pz) = 12

In [17]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_multi_6.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 12
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mo

[default |INFO|model.py:439] [01/30]  loss: 2.208562e-01  D: 6.919593e-01  D2: -1.738548e-05  D3: 3.196724e-03


[default |INFO|model.py:439] [02/30]  loss: 1.629219e-01  D: 6.944335e-01  D2: 2.300406e-04  D3: 5.850781e-03


[default |INFO|model.py:439] [03/30]  loss: 1.510447e-01  D: 6.928213e-01  D2: 2.130159e-04  D3: 5.993537e-03


[default |INFO|model.py:439] [04/30]  loss: 1.134051e-01  D: 6.930627e-01  D2: 3.656165e-04  D3: 5.214536e-03


...

[default |INFO|model.py:608] [47/50]  loss: 4.225319e-02  D: 6.937101e-01  D2: -2.843553e-03  D3: 1.516246e-02
test loss: 4.471836e-02  D: 6.919994e-01  D2: -8.952208e-03  D3: 1.564332e-02


[default |INFO|model.py:608] [48/50]  loss: 4.317601e-02  D: 6.937589e-01  D2: -2.328286e-03  D3: 1.498033e-02
test loss: 4.671790e-02  D: 6.933230e-01  D2: -1.004416e-02  D3: 1.373550e-02


[default |INFO|model.py:608] [49/50]  loss: 4.344628e-02  D: 6.936860e-01  D2: -2.430234e-03  D3: 1.489742e-02
test loss: 4.452644e-02  D: 7.085830e-01  D2: -8.547964e-03  D3: 1.431909e-02


[default |INFO|model.py:608] [50/50]  loss: 4.264607e-02  D: 6.942623e-01  D2: -2.477249e-03  D3: 1.502805e-02
test loss: 4.447070e-02  D: 6.866829e-01  D2: -8.631039e-03  D3: 1.580561e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1150.404s


### dim(Pz) = 13

In [18]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_multi_3.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mo

[default |INFO|model.py:439] [01/30]  loss: 2.179046e-01  D: 6.855279e-01  D2: -2.102280e-04  D3: 2.321051e-03


[default |INFO|model.py:439] [02/30]  loss: 1.630309e-01  D: 6.682811e-01  D2: 4.106204e-05  D3: 7.445428e-03


[default |INFO|model.py:439] [03/30]  loss: 1.509746e-01  D: 6.876002e-01  D2: 2.802674e-04  D3: 1.357403e-02


[default |INFO|model.py:439] [04/30]  loss: 1.227585e-01  D: 6.885006e-01  D2: -3.140100e-04  D3: 5.591855e-03


...

[default |INFO|model.py:608] [47/50]  loss: 3.517105e-02  D: 6.931472e-01  D2: -2.990452e-03  D3: 6.565689e-04
test loss: 3.632020e-02  D: 6.931472e-01  D2: -9.039841e-03  D3: 6.624766e-04


[default |INFO|model.py:608] [48/50]  loss: 3.514872e-02  D: 6.931472e-01  D2: -3.040762e-03  D3: 6.478808e-04
test loss: 3.888404e-02  D: 6.931472e-01  D2: -9.477673e-03  D3: 6.244500e-04


[default |INFO|model.py:608] [49/50]  loss: 3.462086e-02  D: 6.931472e-01  D2: -3.108834e-03  D3: 6.495176e-04
test loss: 3.918003e-02  D: 6.931472e-01  D2: -9.208708e-03  D3: 6.342922e-04


[default |INFO|model.py:608] [50/50]  loss: 3.460872e-02  D: 6.931472e-01  D2: -2.869274e-03  D3: 6.415074e-04
test loss: 3.588849e-02  D: 6.931472e-01  D2: -9.545269e-03  D3: 6.091380e-04


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1128.198s


### dim(Pz) = 14

In [19]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_multi_7.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 14
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mo

[default |INFO|model.py:439] [01/30]  loss: 2.215865e-01  D: 6.908535e-01  D2: 1.380555e-04  D3: 3.542034e-03


[default |INFO|model.py:439] [02/30]  loss: 1.638967e-01  D: 6.871791e-01  D2: 2.719593e-04  D3: 9.667096e-03


[default |INFO|model.py:439] [03/30]  loss: 1.447726e-01  D: 6.957400e-01  D2: 3.514846e-04  D3: 1.092879e-02


[default |INFO|model.py:439] [04/30]  loss: 1.199673e-01  D: 6.918472e-01  D2: -1.896874e-04  D3: 1.129723e-02


...

[default |INFO|model.py:608] [47/50]  loss: 4.124192e-02  D: 6.953624e-01  D2: -2.321634e-03  D3: 1.516957e-02
test loss: 4.366912e-02  D: 7.026416e-01  D2: -8.079729e-03  D3: 1.494085e-02


[default |INFO|model.py:608] [48/50]  loss: 4.067743e-02  D: 6.927532e-01  D2: -2.442109e-03  D3: 1.524635e-02
test loss: 4.075188e-02  D: 6.720753e-01  D2: -8.930702e-03  D3: 1.431504e-02


[default |INFO|model.py:608] [49/50]  loss: 3.862805e-02  D: 6.937257e-01  D2: -2.510575e-03  D3: 1.541994e-02
test loss: 4.090969e-02  D: 6.977248e-01  D2: -8.462639e-03  D3: 1.573429e-02


[default |INFO|model.py:608] [50/50]  loss: 3.960051e-02  D: 6.944792e-01  D2: -2.397726e-03  D3: 1.491819e-02
test loss: 4.366643e-02  D: 7.098733e-01  D2: -8.845587e-03  D3: 1.637162e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1126.148s


### dim(Pz) = 15

In [20]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_multi_8.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 15
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mo

[default |INFO|model.py:439] [01/30]  loss: 2.213209e-01  D: 6.901431e-01  D2: -3.851414e-05  D3: 2.280894e-03


[default |INFO|model.py:439] [02/30]  loss: 1.848994e-01  D: 8.860892e-01  D2: 3.092066e-04  D3: 2.640236e-02


[default |INFO|model.py:439] [03/30]  loss: 1.684354e-01  D: 8.432155e-01  D2: -3.504117e-05  D3: 4.362161e-02


[default |INFO|model.py:439] [04/30]  loss: 1.325362e-01  D: 7.111123e-01  D2: 2.965498e-04  D3: 3.497215e-02


...

[default |INFO|model.py:608] [47/50]  loss: 4.021603e-02  D: 6.936926e-01  D2: -2.328256e-03  D3: 1.511208e-02
test loss: 4.321772e-02  D: 6.884807e-01  D2: -8.946114e-03  D3: 1.612651e-02


[default |INFO|model.py:608] [48/50]  loss: 4.148045e-02  D: 6.954494e-01  D2: -2.466647e-03  D3: 1.530413e-02
test loss: 4.477297e-02  D: 7.019068e-01  D2: -8.687382e-03  D3: 1.545553e-02


[default |INFO|model.py:608] [49/50]  loss: 4.022906e-02  D: 6.921951e-01  D2: -2.492906e-03  D3: 1.515019e-02
test loss: 4.121538e-02  D: 6.822186e-01  D2: -8.679123e-03  D3: 1.494453e-02


[default |INFO|model.py:608] [50/50]  loss: 3.930913e-02  D: 6.938365e-01  D2: -2.746278e-03  D3: 1.514544e-02
test loss: 4.090710e-02  D: 6.849075e-01  D2: -8.925009e-03  D3: 1.355777e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1115.466s


### dim(Pz) = 16

In [21]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_multi_4.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 16
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mo

[default |INFO|model.py:439] [01/30]  loss: 2.167653e-01  D: 6.912729e-01  D2: 2.185837e-04  D3: 2.621127e-03


[default |INFO|model.py:439] [02/30]  loss: 1.438961e-01  D: 6.949053e-01  D2: 1.983054e-04  D3: 5.665135e-03


[default |INFO|model.py:439] [03/30]  loss: 1.150265e-01  D: 6.940255e-01  D2: -2.769343e-04  D3: 4.805530e-03


[default |INFO|model.py:439] [04/30]  loss: 9.797474e-02  D: 6.947446e-01  D2: -1.497857e-04  D3: 9.531889e-03


...

[default |INFO|model.py:608] [47/50]  loss: 4.018771e-02  D: 6.962009e-01  D2: -2.587137e-03  D3: 1.530316e-02
test loss: 4.495661e-02  D: 6.640579e-01  D2: -8.404846e-03  D3: 1.488963e-02


[default |INFO|model.py:608] [48/50]  loss: 3.908106e-02  D: 6.943660e-01  D2: -2.605122e-03  D3: 1.539425e-02
test loss: 4.108879e-02  D: 6.699143e-01  D2: -8.397141e-03  D3: 1.519838e-02


[default |INFO|model.py:608] [49/50]  loss: 3.830953e-02  D: 6.950589e-01  D2: -2.433340e-03  D3: 1.537257e-02
test loss: 4.068222e-02  D: 6.783107e-01  D2: -8.419151e-03  D3: 1.506187e-02


[default |INFO|model.py:608] [50/50]  loss: 3.848079e-02  D: 6.936999e-01  D2: -2.656314e-03  D3: 1.565804e-02
test loss: 3.944802e-02  D: 6.944404e-01  D2: -8.477716e-03  D3: 1.530484e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1148.037s


### tuning lambda

In [5]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_1.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 8.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.147879e-01  D: 6.919310e-01  D2: 1.398404e-04  D3: 2.019894e-03


[default |INFO|model.py:439] [02/30]  loss: 1.632856e-01  D: 7.047300e-01  D2: 2.531528e-05  D3: 1.073396e-02


[default |INFO|model.py:439] [03/30]  loss: 1.483826e-01  D: 7.089861e-01  D2: -3.012180e-04  D3: 2.056507e-02


[default |INFO|model.py:439] [04/30]  loss: 1.330714e-01  D: 7.193454e-01  D2: -2.928066e-04  D3: 2.672474e-02


...

[default |INFO|model.py:608] [46/50]  loss: 4.193620e-02  D: 6.939303e-01  D2: -2.458545e-03  D3: 1.500832e-02
test loss: 4.579367e-02  D: 6.816387e-01  D2: -8.459063e-03  D3: 1.647316e-02


[default |INFO|model.py:608] [47/50]  loss: 4.160254e-02  D: 6.947119e-01  D2: -2.319096e-03  D3: 1.505703e-02
test loss: 4.270051e-02  D: 7.178399e-01  D2: -8.334408e-03  D3: 1.516637e-02


[default |INFO|model.py:608] [48/50]  loss: 4.011563e-02  D: 6.942090e-01  D2: -2.409658e-03  D3: 1.512188e-02
test loss: 4.492872e-02  D: 6.777204e-01  D2: -8.347559e-03  D3: 1.518138e-02


[default |INFO|model.py:608] [49/50]  loss: 4.105920e-02  D: 6.938420e-01  D2: -2.705715e-03  D3: 1.501982e-02
test loss: 4.453400e-02  D: 6.943386e-01  D2: -8.620796e-03  D3: 1.351961e-02


[default |INFO|model.py:608] [50/50]  loss: 4.142768e-02  D: 6.941367e-01  D2: -2.657235e-03  D3: 1.502075e-02
test loss: 4.370571e-02  D: 6.832456e-01  D2: -8.557539e-03  D3: 1.449803e-02
[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1131.460s


In [6]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_2.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 6.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.190408e-01  D: 6.960426e-01  D2: 1.332776e-04  D3: 2.331621e-03


[default |INFO|model.py:439] [02/30]  loss: 1.622394e-01  D: 7.096663e-01  D2: 1.513497e-04  D3: 1.748807e-02


[default |INFO|model.py:439] [03/30]  loss: 1.509812e-01  D: 7.671148e-01  D2: 5.210288e-04  D3: 2.817460e-02


[default |INFO|model.py:439] [04/30]  loss: 1.287242e-01  D: 7.384504e-01  D2: 6.713390e-05  D3: 3.724980e-02


...

[default |INFO|model.py:608] [47/50]  loss: 4.106945e-02  D: 6.946109e-01  D2: -2.457657e-03  D3: 1.510656e-02
test loss: 4.229522e-02  D: 6.819152e-01  D2: -8.963785e-03  D3: 1.472475e-02


[default |INFO|model.py:608] [48/50]  loss: 4.202189e-02  D: 6.947909e-01  D2: -2.148698e-03  D3: 1.516876e-02
test loss: 4.303644e-02  D: 6.922377e-01  D2: -8.766108e-03  D3: 1.458053e-02


[default |INFO|model.py:608] [49/50]  loss: 4.146318e-02  D: 6.952079e-01  D2: -2.172003e-03  D3: 1.522591e-02
test loss: 4.488505e-02  D: 6.820229e-01  D2: -8.720922e-03  D3: 1.497433e-02


[default |INFO|model.py:608] [50/50]  loss: 4.184988e-02  D: 6.938533e-01  D2: -2.016460e-03  D3: 1.513321e-02
test loss: 4.419583e-02  D: 6.772074e-01  D2: -8.634996e-03  D3: 1.474235e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1105.672s


In [7]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_3.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 4.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.160447e-01  D: 6.938096e-01  D2: -6.653468e-06  D3: 3.158499e-03


[default |INFO|model.py:439] [02/30]  loss: 1.571731e-01  D: 6.964058e-01  D2: -2.601290e-04  D3: 6.397586e-03


[default |INFO|model.py:439] [03/30]  loss: 1.375414e-01  D: 6.954283e-01  D2: -8.460681e-06  D3: 7.519316e-03


[default |INFO|model.py:439] [04/30]  loss: 1.188226e-01  D: 7.076089e-01  D2: 2.169371e-04  D3: 1.561850e-02


...

[default |INFO|model.py:608] [46/50]  loss: 4.089390e-02  D: 6.941754e-01  D2: -1.943447e-03  D3: 1.500206e-02
test loss: 4.229178e-02  D: 6.987497e-01  D2: -8.095036e-03  D3: 1.529201e-02


[default |INFO|model.py:608] [47/50]  loss: 4.074470e-02  D: 6.952282e-01  D2: -2.669304e-03  D3: 1.490813e-02
test loss: 4.381080e-02  D: 6.849955e-01  D2: -9.043598e-03  D3: 1.436685e-02


[default |INFO|model.py:608] [48/50]  loss: 4.045839e-02  D: 6.948594e-01  D2: -2.887600e-03  D3: 1.494973e-02
test loss: 4.196530e-02  D: 6.879707e-01  D2: -9.030724e-03  D3: 1.495193e-02


[default |INFO|model.py:608] [49/50]  loss: 4.042454e-02  D: 6.951937e-01  D2: -2.518673e-03  D3: 1.495862e-02
test loss: 4.121501e-02  D: 6.956287e-01  D2: -8.433733e-03  D3: 1.507089e-02


[default |INFO|model.py:608] [50/50]  loss: 3.979176e-02  D: 6.961275e-01  D2: -2.695045e-03  D3: 1.488577e-02
test loss: 4.397608e-02  D: 6.848547e-01  D2: -8.929062e-03  D3: 1.382537e-02
[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1105.188s


In [8]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_4.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 8.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.264812e-01  D: 1.177677e+00  D2: 2.960396e-04  D3: 6.957195e-03


[default |INFO|model.py:439] [02/30]  loss: 2.185288e-01  D: 3.446922e+00  D2: -8.828004e-05  D3: 3.909224e-02


[default |INFO|model.py:439] [03/30]  loss: 2.085138e-01  D: 1.927437e+00  D2: 2.464263e-04  D3: 4.241351e-02


[default |INFO|model.py:439] [04/30]  loss: 1.855323e-01  D: 9.390214e-01  D2: 5.760511e-05  D3: 4.349247e-02


...

[default |INFO|model.py:608] [47/50]  loss: 4.026491e-02  D: 6.935030e-01  D2: -2.177844e-03  D3: 1.489040e-02
test loss: 4.260521e-02  D: 6.916272e-01  D2: -8.555527e-03  D3: 1.541123e-02


[default |INFO|model.py:608] [48/50]  loss: 4.098516e-02  D: 6.935842e-01  D2: -1.914612e-03  D3: 1.488522e-02
test loss: 4.209828e-02  D: 6.892556e-01  D2: -8.682575e-03  D3: 1.504929e-02


[default |INFO|model.py:608] [49/50]  loss: 4.040532e-02  D: 6.941485e-01  D2: -1.823111e-03  D3: 1.478775e-02
test loss: 4.354765e-02  D: 6.993263e-01  D2: -8.774614e-03  D3: 1.396145e-02


[default |INFO|model.py:608] [50/50]  loss: 4.091566e-02  D: 6.923230e-01  D2: -2.091260e-03  D3: 1.478525e-02
test loss: 4.279715e-02  D: 6.980160e-01  D2: -8.654242e-03  D3: 1.581651e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1107.137s


In [9]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_5.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 6.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.382518e-01  D: 1.883518e+00  D2: 6.468423e-04  D3: 1.089164e-02


[default |INFO|model.py:439] [02/30]  loss: 1.993987e-01  D: 2.398044e+00  D2: 9.431521e-05  D3: 4.324041e-02


[default |INFO|model.py:439] [03/30]  loss: 1.657807e-01  D: 1.038453e+00  D2: -1.994371e-04  D3: 4.344532e-02


[default |INFO|model.py:439] [04/30]  loss: 1.361481e-01  D: 7.053158e-01  D2: 3.396432e-04  D3: 3.807786e-02


...

[default |INFO|model.py:608] [47/50]  loss: 4.100887e-02  D: 6.958148e-01  D2: -1.608005e-03  D3: 1.433168e-02
test loss: 4.453709e-02  D: 6.882707e-01  D2: -9.640150e-03  D3: 1.163358e-02


[default |INFO|model.py:608] [48/50]  loss: 4.309405e-02  D: 6.936972e-01  D2: -1.764261e-03  D3: 1.413166e-02
test loss: 4.492431e-02  D: 6.930954e-01  D2: -8.505316e-03  D3: 1.338999e-02


[default |INFO|model.py:608] [49/50]  loss: 4.326229e-02  D: 6.928248e-01  D2: -1.305312e-03  D3: 1.421513e-02
test loss: 4.434418e-02  D: 6.974877e-01  D2: -8.977671e-03  D3: 1.281943e-02


[default |INFO|model.py:608] [50/50]  loss: 4.259026e-02  D: 6.935053e-01  D2: -1.389937e-03  D3: 1.434393e-02
test loss: 4.664583e-02  D: 7.079908e-01  D2: -8.433495e-03  D3: 1.316431e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1108.190s


In [10]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_6.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 4.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.390359e-01  D: 1.353100e+00  D2: -1.356125e-04  D3: 1.076934e-02


[default |INFO|model.py:439] [02/30]  loss: 1.943016e-01  D: 1.542686e+00  D2: 7.333755e-06  D3: 4.418679e-02


[default |INFO|model.py:439] [03/30]  loss: 1.614876e-01  D: 8.103664e-01  D2: -1.743412e-04  D3: 4.116115e-02


[default |INFO|model.py:439] [04/30]  loss: 1.315398e-01  D: 6.820475e-01  D2: 1.055272e-04  D3: 2.900638e-02


...

[default |INFO|model.py:608] [47/50]  loss: 4.114252e-02  D: 6.939790e-01  D2: -3.790538e-05  D3: 1.342532e-02
test loss: 4.148926e-02  D: 6.877470e-01  D2: -7.783594e-03  D3: 1.269712e-02


[default |INFO|model.py:608] [48/50]  loss: 4.054332e-02  D: 6.943943e-01  D2: -7.933458e-05  D3: 1.369930e-02
test loss: 4.192774e-02  D: 6.983444e-01  D2: -7.637711e-03  D3: 1.189809e-02


[default |INFO|model.py:608] [49/50]  loss: 4.037407e-02  D: 6.932101e-01  D2: -4.020055e-05  D3: 1.395016e-02
test loss: 4.229978e-02  D: 6.807376e-01  D2: -7.301874e-03  D3: 1.432496e-02


[default |INFO|model.py:608] [50/50]  loss: 4.072210e-02  D: 6.941165e-01  D2: -5.898476e-05  D3: 1.386315e-02
test loss: 4.456033e-02  D: 7.062627e-01  D2: -7.061539e-03  D3: 1.623918e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1135.506s


In [11]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_7.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 8.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.198170e-01  D: 6.895075e-01  D2: -3.305944e-04  D3: 2.498805e-03


[default |INFO|model.py:439] [02/30]  loss: 1.927533e-01  D: 9.855057e-01  D2: 1.373704e-04  D3: 3.385768e-02


[default |INFO|model.py:439] [03/30]  loss: 1.625742e-01  D: 7.812976e-01  D2: -1.124525e-04  D3: 4.479396e-02


[default |INFO|model.py:439] [04/30]  loss: 1.262330e-01  D: 7.052576e-01  D2: 1.368523e-05  D3: 3.643669e-02


...

[default |INFO|model.py:608] [47/50]  loss: 3.962495e-02  D: 6.941624e-01  D2: -2.553910e-03  D3: 1.528350e-02
test loss: 4.225646e-02  D: 7.004759e-01  D2: -8.632336e-03  D3: 1.477417e-02


[default |INFO|model.py:608] [48/50]  loss: 3.946844e-02  D: 6.925365e-01  D2: -2.560032e-03  D3: 1.562506e-02
test loss: 4.244481e-02  D: 7.034889e-01  D2: -9.176664e-03  D3: 1.582988e-02


[default |INFO|model.py:608] [49/50]  loss: 3.854907e-02  D: 6.948864e-01  D2: -2.469810e-03  D3: 1.540040e-02
test loss: 4.045809e-02  D: 6.960839e-01  D2: -8.453188e-03  D3: 1.640735e-02


[default |INFO|model.py:608] [50/50]  loss: 3.834476e-02  D: 6.924609e-01  D2: -2.751973e-03  D3: 1.549389e-02
test loss: 4.195360e-02  D: 6.980790e-01  D2: -8.219299e-03  D3: 1.507743e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1144.913s


In [12]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_8.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 6.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.360844e-01  D: 1.198255e+00  D2: 3.410800e-04  D3: 1.311395e-02


[default |INFO|model.py:439] [02/30]  loss: 2.013526e-01  D: 1.721865e+00  D2: 7.328987e-06  D3: 4.412672e-02


[default |INFO|model.py:439] [03/30]  loss: 1.697547e-01  D: 1.002685e+00  D2: -7.580121e-05  D3: 4.196495e-02


[default |INFO|model.py:439] [04/30]  loss: 1.408148e-01  D: 8.021335e-01  D2: -7.336775e-05  D3: 3.585493e-02


...

[default |INFO|model.py:608] [47/50]  loss: 3.755673e-02  D: 6.930348e-01  D2: -2.718528e-03  D3: 1.544777e-02
test loss: 4.010991e-02  D: 6.999551e-01  D2: -8.400526e-03  D3: 1.674614e-02


[default |INFO|model.py:608] [48/50]  loss: 3.655576e-02  D: 6.936029e-01  D2: -2.635606e-03  D3: 1.535072e-02
test loss: 3.923739e-02  D: 6.928608e-01  D2: -9.133177e-03  D3: 1.549262e-02


[default |INFO|model.py:608] [49/50]  loss: 3.684204e-02  D: 6.920731e-01  D2: -2.912299e-03  D3: 1.545650e-02
test loss: 3.935200e-02  D: 6.885248e-01  D2: -9.361534e-03  D3: 1.516901e-02


[default |INFO|model.py:608] [50/50]  loss: 3.694635e-02  D: 6.935882e-01  D2: -2.750703e-03  D3: 1.552936e-02
test loss: 3.921870e-02  D: 6.935929e-01  D2: -8.730545e-03  D3: 1.641424e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1113.123s


In [13]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_9.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 4.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.341674e-01  D: 1.785493e+00  D2: -1.753457e-04  D3: 1.573985e-02


[default |INFO|model.py:439] [02/30]  loss: 1.984751e-01  D: 1.942297e+00  D2: 5.000512e-04  D3: 4.629831e-02


[default |INFO|model.py:439] [03/30]  loss: 1.620716e-01  D: 9.144113e-01  D2: -1.592716e-04  D3: 4.463331e-02


[default |INFO|model.py:439] [04/30]  loss: 1.300340e-01  D: 7.236367e-01  D2: -1.879946e-04  D3: 3.913685e-02


...

[default |INFO|model.py:608] [46/50]  loss: 3.411554e-02  D: 6.930982e-01  D2: -3.034507e-03  D3: 1.613354e-02
test loss: 3.690148e-02  D: 6.933016e-01  D2: -8.794737e-03  D3: 1.580455e-02


[default |INFO|model.py:608] [47/50]  loss: 3.458081e-02  D: 6.930421e-01  D2: -3.017616e-03  D3: 1.599524e-02
test loss: 3.667107e-02  D: 6.959231e-01  D2: -8.172331e-03  D3: 1.524766e-02


[default |INFO|model.py:608] [48/50]  loss: 3.420912e-02  D: 6.932857e-01  D2: -2.860856e-03  D3: 1.611221e-02
test loss: 3.776311e-02  D: 6.934709e-01  D2: -8.863745e-03  D3: 1.616831e-02


[default |INFO|model.py:608] [49/50]  loss: 3.424533e-02  D: 6.932968e-01  D2: -2.940806e-03  D3: 1.603219e-02
test loss: 3.644367e-02  D: 6.923386e-01  D2: -9.153910e-03  D3: 1.536472e-02


[default |INFO|model.py:608] [50/50]  loss: 3.390093e-02  D: 6.932244e-01  D2: -2.890873e-03  D3: 1.608424e-02
test loss: 3.828722e-02  D: 6.884397e-01  D2: -8.985109e-03  D3: 1.578392e-02
[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1107.801s


In [14]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_multi_3.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 10.0
[default |INFO|_base_mo

[default |INFO|model.py:439] [01/30]  loss: 2.373508e-01  D: 1.387653e+00  D2: 3.027407e-04  D3: 9.871309e-03


[default |INFO|model.py:439] [02/30]  loss: 1.931818e-01  D: 2.038660e+00  D2: -3.246435e-04  D3: 4.251094e-02


[default |INFO|model.py:439] [03/30]  loss: 1.578213e-01  D: 8.681113e-01  D2: 2.043263e-04  D3: 3.895371e-02


[default |INFO|model.py:439] [04/30]  loss: 1.296889e-01  D: 8.071796e-01  D2: 2.763033e-04  D3: 3.299734e-02


...

[default |INFO|model.py:608] [47/50]  loss: 4.223218e-02  D: 6.946568e-01  D2: -2.216487e-03  D3: 1.500687e-02
test loss: 4.455148e-02  D: 7.042476e-01  D2: -8.662930e-03  D3: 1.488074e-02


[default |INFO|model.py:608] [48/50]  loss: 4.284279e-02  D: 6.949178e-01  D2: -2.237384e-03  D3: 1.490126e-02
test loss: 4.425054e-02  D: 6.934768e-01  D2: -9.391813e-03  D3: 1.401118e-02


[default |INFO|model.py:608] [49/50]  loss: 4.103981e-02  D: 6.925625e-01  D2: -2.540909e-03  D3: 1.505188e-02
test loss: 4.311249e-02  D: 6.815178e-01  D2: -8.503027e-03  D3: 1.527857e-02


[default |INFO|model.py:608] [50/50]  loss: 4.203279e-02  D: 6.936384e-01  D2: -2.315871e-03  D3: 1.511717e-02
test loss: 4.296490e-02  D: 6.887027e-01  D2: -8.208389e-03  D3: 1.533560e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1125.447s


In [16]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_10.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 2.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.265681e-01  D: 9.188333e-01  D2: -1.646868e-04  D3: 1.291058e-02


[default |INFO|model.py:439] [02/30]  loss: 1.953600e-01  D: 1.445519e+00  D2: -6.393115e-05  D3: 4.365167e-02


[default |INFO|model.py:439] [03/30]  loss: 1.568221e-01  D: 8.270881e-01  D2: 2.313566e-04  D3: 4.023947e-02


[default |INFO|model.py:439] [04/30]  loss: 1.353779e-01  D: 7.133291e-01  D2: 1.042461e-04  D3: 3.594749e-02


...

[default |INFO|model.py:608] [47/50]  loss: 3.312246e-02  D: 6.931145e-01  D2: -2.769856e-03  D3: 1.652759e-02
test loss: 3.675756e-02  D: 6.933705e-01  D2: -8.990040e-03  D3: 1.656419e-02


[default |INFO|model.py:608] [48/50]  loss: 3.288035e-02  D: 6.932451e-01  D2: -2.966407e-03  D3: 1.660923e-02
test loss: 3.659460e-02  D: 6.949579e-01  D2: -8.994703e-03  D3: 1.659526e-02


[default |INFO|model.py:608] [49/50]  loss: 3.329409e-02  D: 6.931595e-01  D2: -2.853808e-03  D3: 1.665218e-02
test loss: 3.654341e-02  D: 6.950488e-01  D2: -9.456358e-03  D3: 1.674014e-02


[default |INFO|model.py:608] [50/50]  loss: 3.294597e-02  D: 6.931752e-01  D2: -2.693086e-03  D3: 1.652754e-02
test loss: 3.839206e-02  D: 6.909794e-01  D2: -8.900671e-03  D3: 1.729090e-02


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1143.283s


In [17]:
cfg = configparser.ConfigParser()
cfg.read('./config/train_config_sswae_hsic_13_11.cfg')

mm = cur_model(cfg, log, device = device)

mm.embed_data.load_state_dict(torch.load('./result/embed_data_weight.pt'))
mm.embed_condition.load_state_dict(torch.load('./result/embed_condition_weight.pt'))

mm.train()

[default |INFO|_base_model.py:25] ------------------------------------------------------------
[default |INFO|_base_model.py:27] model_name : SSWAE_HSIC_MNIST
[default |INFO|_base_model.py:27] train_data : MNIST
[default |INFO|_base_model.py:27] train_data_label : True
[default |INFO|_base_model.py:27] z_sampler : gaus
[default |INFO|_base_model.py:27] z_dim : 13
[default |INFO|_base_model.py:27] y_sampler : multinomial
[default |INFO|_base_model.py:27] y_dim : 10
[default |INFO|_base_model.py:27] portion : 1.0
[default |INFO|_base_model.py:27] encoder_pretrain : True
[default |INFO|_base_model.py:27] encoder_pretrain_step : 30
[default |INFO|_base_model.py:27] lr : 5e-4
[default |INFO|_base_model.py:27] beta1 : 0.5
[default |INFO|_base_model.py:27] lr_adv : 2e-4
[default |INFO|_base_model.py:27] beta1_adv : 0.5
[default |INFO|_base_model.py:27] lambda : 10.0
[default |INFO|_base_model.py:27] lambda_mmd : 10.0
[default |INFO|_base_model.py:27] lambda_hsic : 1.0
[default |INFO|_base_mod

[default |INFO|model.py:439] [01/30]  loss: 2.057880e-01  D: 6.941132e-01  D2: -3.331359e-04  D3: 6.170325e-03


[default |INFO|model.py:439] [02/30]  loss: 1.223419e-01  D: 6.931468e-01  D2: -1.060724e-04  D3: 3.078859e-03


[default |INFO|model.py:439] [03/30]  loss: 8.682332e-02  D: 6.931472e-01  D2: 7.093430e-05  D3: 3.082063e-03


[default |INFO|model.py:439] [04/30]  loss: 6.300516e-02  D: 6.931472e-01  D2: -1.312256e-05  D3: 3.257528e-03


...

[default |INFO|model.py:608] [47/50]  loss: 3.120313e-02  D: 6.931474e-01  D2: -2.867400e-03  D3: 1.979262e-03
test loss: 3.403848e-02  D: 6.931474e-01  D2: -7.694225e-03  D3: 1.987013e-03


[default |INFO|model.py:608] [48/50]  loss: 3.077379e-02  D: 6.931474e-01  D2: -2.738687e-03  D3: 1.984333e-03
test loss: 3.517407e-02  D: 6.931474e-01  D2: -8.158541e-03  D3: 1.980060e-03


[default |INFO|model.py:608] [49/50]  loss: 3.048705e-02  D: 6.931474e-01  D2: -2.769353e-03  D3: 1.941925e-03
test loss: 3.249829e-02  D: 6.931474e-01  D2: -8.379345e-03  D3: 1.952717e-03


[default |INFO|model.py:608] [50/50]  loss: 3.088429e-02  D: 6.931474e-01  D2: -3.051146e-03  D3: 1.935680e-03
test loss: 3.360467e-02  D: 6.931474e-01  D2: -8.308163e-03  D3: 1.915323e-03


[default |INFO|model.py:682] Training Finished!
[default |INFO|model.py:683] Elapsed time: 1215.005s
